In [1]:
import h5py
import tensorflow as tf
import numpy as np

from tensorflow.contrib.rnn import LSTMCell
from tensorflow.contrib.rnn import MultiRNNCell

In [2]:
def conv_layer(indata, out_channels, batch_norm=True, k=1, padding='SAME', stride=1):
    strides = [1, stride, stride, 1]
    in_channels = indata.get_shape()[-1]
    shape = [1, k, in_channels, out_channels]
    W = tf.get_variable('weights', shape=shape)
    conv_out = tf.nn.conv2d(indata, W, strides=strides, padding=padding)
    if batch_norm:
        with tf.variable_scope('batch_normalization'):
            mean, variance = tf.nn.moments(conv_out, [0, 1, 2])
            scale = tf.get_variable('scale', shape=[out_channels])
            offset = tf.get_variable('offset', shape=[out_channels])
            conv_out = tf.nn.batch_normalization(conv_out, 
                                                 mean=mean, 
                                                 variance=variance, 
                                                 scale=scale, 
                                                 offset=offset,
                                                 variance_epsilon=1e-5)
    return conv_out

def residual_layer(indata, out_channels, batch_norm=True, stride=1):
    in_channel = indata.get_shape()[-1]
    with tf.variable_scope('conv1'):
        conv1 = conv_layer(indata, out_channels, batch_norm)
    
    with tf.variable_scope('conv2'):
        with tf.variable_scope('a'):
            conv2 = conv_layer(indata, out_channels)
            conv2 = tf.nn.relu(conv2)

        with tf.variable_scope('b'):
            conv2 = conv_layer(conv2, out_channels, k=3)
            conv2 = tf.nn.relu(conv2)

        with tf.variable_scope('c'):
            conv2 = conv_layer(conv2, out_channels)
            conv2 = tf.nn.relu(conv2)
    
    return tf.nn.relu(conv1 + conv2)

In [3]:
from fast5_input import fast5batches

batch_size = 200
segment_length = 400

f5b = fast5batches(batch_size=batch_size, 
                   segment_length=segment_length, 
                   fast5dir='../chiron-otrain/pass', 
                   training=True, 
                   test_ratio=.2,
                   overlap=30)

checking files: 100%|██████████| 34383/34383 [00:16<00:00, 2097.42it/s]

29849 files have labels, 23880 for training and 5969 for testing


In [4]:
f5b.next_batch(test=False, fill=True)
f5b.next_batch(test=True, fill=True)

200it [00:35,  3.08it/s]
200it [00:31,  3.39it/s]


(array([], shape=(0, 400, 3), dtype=float64),
 array([], dtype=int64),
 Labels(indices=array([], shape=(0, 2), dtype=int32), values=array([], dtype=int64)),
 array([], dtype=[('name', 'S142'), ('i', '<u4'), ('total', '<u4')]))

In [5]:

num_features = f5b.raw_signals.shape[2]

neurons_per_layer = 100
layers = 4
class_n = 5

print(num_features)

3


In [6]:
x = tf.placeholder(tf.float32, shape=[batch_size, segment_length, num_features], name = 'x')
sequence_length = tf.placeholder(tf.int32, shape=[batch_size], name = 'sequence_length')

In [7]:
in_channels = num_features

net = tf.reshape(x, [batch_size, 1, segment_length, in_channels])
out_channel = 256

for i in range(3):
    with tf.variable_scope('residual_layer_'+str(i)):
        net = residual_layer(net, out_channel, batch_norm=i==0)

In [8]:
conv_all = tf.reshape(net, [batch_size, segment_length, out_channel])

In [9]:
##########################################
## first RNN layers ######################
##########################################

outputs, output_states = tf.nn.bidirectional_dynamic_rnn(
    cell_fw = MultiRNNCell([LSTMCell(neurons_per_layer) for _ in range(layers)]), 
    cell_bw = MultiRNNCell([LSTMCell(neurons_per_layer) for _ in range(layers)]), 
    inputs = conv_all, 
    sequence_length = sequence_length,
    dtype = tf.float32
)

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


In [10]:
with tf.name_scope('collapse'):

    outputs = tf.concat(outputs, 2)
    outputs = tf.reshape(outputs, [batch_size, segment_length, 2, neurons_per_layer], name = 'outputs')

    W_collapse = tf.get_variable('W_collapse', shape = [2, neurons_per_layer])
    b_collapse = tf.get_variable('b_callapse', shape = [neurons_per_layer])

    collapsed = tf.multiply(outputs, W_collapse)
    collapsed = tf.reduce_sum(collapsed, axis=2)
    collapsed = tf.nn.bias_add(collapsed, b_collapse)
    collapsed = tf.reshape(collapsed, [batch_size*segment_length,neurons_per_layer], name = 'collapsed')

with tf.name_scope('FF'):
    
    W_last = tf.get_variable('W_last', shape = [neurons_per_layer, class_n])
    b_last = tf.get_variable('b_last', shape = [class_n])

    logits = tf.matmul(collapsed, W_last)
    logits = tf.add(logits, b_last)
    logits = tf.reshape(logits, [batch_size, segment_length, class_n], name="logits")
logits

<tf.Tensor 'FF/logits:0' shape=(200, 400, 5) dtype=float32>

In [11]:
with tf.name_scope('labels'):
    indices = tf.placeholder(tf.int64, name='indices')
    y = tf.placeholder(tf.int32, name='values')
    dense_shape = (batch_size, segment_length)

    labels = tf.SparseTensor(indices = indices, 
                             values = y, 
                             dense_shape = dense_shape
                            )
with tf.name_scope('loss'):
    loss = tf.reduce_mean(
        tf.nn.ctc_loss(labels = labels, 
                       inputs = logits, 
                       sequence_length = sequence_length, 
                       time_major=False
                      )
    )

In [12]:
#max_steps = 20000
#init_rate = 1e-3
global_step = tf.get_variable('global_step', trainable=False, shape=(),
                              dtype=tf.int32,
                              initializer=tf.zeros_initializer())
#boundaries = [21000, 31000]
#lr_values = [1e-4, 1e-5, 1e-6]
#learning_rate = tf.train.piecewise_constant(global_step, boundaries, lr_values)
learning_rate = tf.placeholder(tf.float32, name='learning_rate')

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

step = optimizer.minimize(loss, global_step = global_step)

In [13]:
with tf.name_scope('error'):
    logits_transposed = tf.transpose(logits, perm=[1, 0, 2])
    #print(logits)
    #decoded, neg_sum_logits = tf.nn.ctc_greedy_decoder(logits_transposed, sequence_length, merge_repeated=True)
    decoded, neg_sum_logits = tf.nn.ctc_beam_search_decoder(
                logits_transposed,
                sequence_length,
                merge_repeated=False,
                top_paths=1,
                beam_width=30)

    first_path = decoded[0]
    edit_d = tf.edit_distance(tf.to_int32(first_path), labels, normalize=True)
    error = tf.reduce_mean(edit_d, axis=0)
    #err_4merge = tf.summary.scalar('Error_rate', error)
    error

In [14]:
save_path = 'log/save'

#loss_4merge = tf.summary.scalar('loss', loss)
#lr_4merge = tf.summary.scalar('learning_rate', learning_rate)

tf.summary.histogram('W_last', W_last)
tf.summary.histogram('b_last', b_last)

summary_train = tf.summary.merge([tf.summary.scalar('learning_rate', learning_rate), 
                                  tf.summary.scalar('loss', loss)])
summary_test = tf.summary.merge([tf.summary.scalar('Error_rate', error)])


#summary_all = tf.summary.merge_all()


In [15]:
import datetime
from tqdm import tqdm


if True:
    i = 1
    name = 'withall_exp-2_'+str(i)
    #retrain = True
    summary_path = 'logs/summary/' + name
    lr_val = 10**-2

    with tf.Session() as sess:
        saver = tf.train.Saver()
        summary_writer = tf.summary.FileWriter(summary_path, sess.graph)
        try:
            saver.restore(sess, './logs/save/{}.ckpt'.format(name))
        except ValueError:
            sess.run(tf.global_variables_initializer())
        #for _ in tqdm(range(2500)):
        with tqdm() as bar:
            while True:
                X, ln, Y, fi = f5b.next_batch(test=False)
                feed_dict = {x: X, 
                             sequence_length: ln, 
                             indices: Y.indices, 
                             y: Y.values,
                             learning_rate: lr_val}
                summary, _, gs_val, loss_val = sess.run([summary_train, step, global_step, loss], feed_dict = feed_dict)
                summary_writer.add_summary(summary, global_step=gs_val)
                summary_writer.flush()
                bar.n = gs_val
                bar.refresh()
                if gs_val%10==0:
                    X, ln, Y, fi = f5b.next_batch(test=True)
                    feed_dict = {x: X, 
                                 sequence_length: ln, 
                                 indices: Y.indices, 
                                 y: Y.values,}
                    summary, err_val, gs_val = sess.run([summary_test, error, global_step], feed_dict = feed_dict)
                    summary_writer.add_summary(summary, global_step=gs_val)
                    summary_writer.flush()
                    saver.save(sess, './logs/save/{}.ckpt'.format(name))

5830it [5:48:33,  3.59s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

10312it [10:17:41,  3.59s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

14943it [14:55:54,  3.60s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=

KeyboardInterrupt: 

In [ ]:
from  glob import glob
glob('./logs/save/withouttime_exp_-2*')

In [ ]:
saver = tf.train.Saver()

if True:
    with tf.Session() as sess:
        name = 'withouttime_exp_-2_5'
        saver.restore(sess, './logs/save/{}.ckpt-2500'.format(name))
        temp = f5b.next_batch(test=False)
        rs, ln, vals, ixs, fi = temp
        feed_dict = {x:rs, 
                     sequence_length:ln}
        #ed, fp, lgs = sess.run([edit_d, first_path, logits], feed_dict=feed_dict)
        fp, lgs = sess.run([first_path, logits], feed_dict=feed_dict)

In [ ]:
fp

In [ ]:
fp.values[fp.indices.T[0]==0]

In [ ]:
import matplotlib.pylab as plt
import jupyterthemes as jt
%matplotlib inline
jt.jtplot.style('onedork')
import pylab
pylab.rcParams['figure.figsize'] = (15,5)

In [ ]:
import h5py
rs_list = []

for ff in fi:
    with h5py.File(ff['name'].decode(),'r') as input_data:
        for read_name in input_data['Raw/Reads']:
            s = segment_length*ff['i']
            e = segment_length*(1+ff['i'])
            rs_list.append(input_data['Raw/Reads'][read_name]['Signal'][s:e])

In [ ]:
import random




#plt.plot(raw_signals)
#plt.step(x=np.arange(end-start),y=raw_signals[start:end])

In [ ]:
#lgs -= np.min(lgs)
lgs = np.exp(lgs)
lgs /= np.sum(lgs,2)[:,:,None]

In [ ]:
i = random.randint(0,len(rs_list)-1)
raw_signals = rs_list[i]
l=90
start = random.randint(0,len(raw_signals)-1-l)
end = start+l

fig, axes = plt.subplots(2, sharex='all')



im = axes[1].imshow(lgs[0,start:end].T, aspect='auto')
axes[1].grid(False)

ix = [['A','C','G','T',''][b] for b in lgs[0,start:end].argmax(axis=1)]

axes[1].set_yticks(np.arange(5))
axes[1].set_yticklabels(['A','C','G','T',r'$\epsilon$'])

fig.colorbar(im, ax=axes[1], orientation='horizontal')




axes[0].plot(raw_signals[start:end])
axes[0].set_xticks(np.arange(end-start))
axes[0].set_xticklabels(ix)

for x,y,s in zip(np.arange(end-start), raw_signals[start:end], ix):
    axes[0].text(x=x,y=y,s=s)


jt.jtplot.style(None)
plt.rcParams['figure.figsize'] = (15,5)
plt.savefig("logits.png", bbox_inches='tight')
plt.show()

In [ ]:
print(vals[ixs.T[0]==0])

In [ ]:
print(fp.values[fp.indices.T[0]==0])

In [ ]:
ix_n = lgs[0].argmax(axis=1)
print(ix_n[ix_n!=4])

In [ ]:
def collapse(a):
    r = []
    last_el = None
    for el in a:
        if el==last_el:
            continue
        else:
            r.append(el)
            last_el = el
    return np.array(r)
def rm_4(a):
    return a[a!=4]


a = lgs[0].argmax(axis=1)
#print(a)
print(rm_4(collapse(a)))